# Comparing Temperatures and Luminosities Between Analyses

This notebook measures global temperatures and luminosities for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of galaxy cluster masses, temperatures and luminosities play an important role in the construction of X-ray mass-observable scaling relations - thus they must also be shown to be consistent with previous work. 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM

from xga.samples import ClusterSample

## Reading in Sample Files and Declaring XGA ClusterSamples

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

In [3]:
xcs3p = pd.read_csv("sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")
# TODO OBVIOUSLY REMOVE
xcs3p = xcs3p[:10]

xcs3p.columns

Index(['MEM_MATCH_ID', 'xapa_ra', 'xapa_dec', 'rm_ra', 'rm_dec', 'z',
       'outer_rad', 'Tx', 'Tx-', 'Tx+', 'Lx52', 'Lx52-', 'Lx52+', 'Lx', 'Lx-',
       'Lx+', 'richness', 'richness_err'],
      dtype='object')

In [ ]:
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
zs = xcs3p['z'].values
ns = xcs3p['name'].values

xcs_srcs = ClusterSample()

In [ ]:
xxlgc100 = pd.read_csv("sample_files/xxl_gc100.csv")

In [ ]:
locuss = pd.read_csv("sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})

In [ ]:
lovisari = pd.read_csv("sample_files/lovisari_planck_clusters.csv")